In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '7'
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import keras
import random

In [4]:
# %load model.py
from keras.models import Model
from keras.layers import Input,Conv1D, Dense, MaxPool1D, Activation, AvgPool1D,GlobalAveragePooling1D
from keras.layers import Flatten, Add, Concatenate, Dropout, BatchNormalization
from keras.regularizers import l2

def ResidualBlock(filters,kernel_size,strides,pool_size,inputs, l_2=0.0, activation='relu', kernel_initializer='he_normal'):
    path1 = MaxPool1D(pool_size=pool_size, padding = 'same', strides = strides)(inputs)
    
    path2 = BatchNormalization()(inputs)
    path2 = Activation(activation=activation)(path2)
    path2 = Conv1D(filters = filters, kernel_size = kernel_size, strides = strides, padding = 'same',
                   kernel_regularizer = l2(l_2),
                   kernel_initializer = kernel_initializer)(path2)
    path2 = BatchNormalization()(path2)
    path2 = Activation(activation=activation)(path2)
    path2 = Conv1D(filters = filters, kernel_size = kernel_size, strides = 1, padding = 'same', 
                   kernel_regularizer = l2(l_2),
                   kernel_initializer = kernel_initializer)(path2)
    path2 = Add()([path2, path1])
    return path2

def build_model(length=300, n_channel=2, n_classes=2, filters=64, kernel_size=3, layers = 10,
                activation='relu',kernel_initializer = 'he_normal', l_2=0.0):    
    sig_inp =  Input(shape=(length, n_channel))  
    inp = Conv1D(filters=filters, kernel_size=kernel_size, strides=1, padding="same", 
                 kernel_regularizer=l2(l_2))(sig_inp)
    inp = BatchNormalization()(inp)
    inp = Activation(activation=activation)(inp)
    inp_max = MaxPool1D(pool_size=2)(inp)

    l1 = Conv1D(filters=filters, kernel_size=kernel_size, strides=2, padding="same",
                kernel_regularizer=l2(l_2))(inp)
    l1 = BatchNormalization()(l1)
    l1 = Activation(activation=activation)(l1)
    l1 = Conv1D(filters=filters, kernel_size=kernel_size, strides=1, padding="same",
                kernel_regularizer=l2(l_2))(l1)

    new_inp = Add()([l1,inp_max])

    for i in range(layers):
    # every alternate residual block subsample its input by a factor of 2
        if i % 2 == 1:
            pool_size = 2
            strides = 2
        else:
            pool_size = 1
            strides = 1
        # incremented filters    
        if i % 4 == 3:
            filters = 64*int(i//4 + 2)
            new_inp = Conv1D(filters = filters, kernel_size = kernel_size, strides = 1, padding = 'same',
                             kernel_regularizer=l2(l_2),
                             kernel_initializer = kernel_initializer)(new_inp)
        new_inp = ResidualBlock(filters,kernel_size,strides,pool_size,new_inp, l_2=l_2)

    new_inp = GlobalAveragePooling1D()(new_inp)
    new_inp = BatchNormalization()(new_inp)
    new_inp = Dense(128, kernel_regularizer=l2(l_2))(new_inp) 
    new_inp = BatchNormalization()(new_inp)
    new_inp = Activation(activation=activation)(new_inp)
    out = Dense(n_classes, activation='softmax', kernel_regularizer=l2(l_2))(new_inp)
    
    model = Model(inputs=[sig_inp],outputs=[out])
    return model



In [5]:
# %load utils.py
import os
import keras
import random
import itertools
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences

MEAN_B, STD_B = 138.712, 16.100
MEAN_M, STD_M =  36.346, 25.224

def Probability(a):
    def prob():
        return random.uniform(a=0, b=1) > a
    return prob

def DA_RandomNoise(x):
    noise = np.array([[random.gauss(mu=0, sigma=0.01), 
                       random.gauss(mu=0, sigma=0.01)] for _ in range(x.shape[0])], dtype=np.float32)
    return x + noise

def DA_Shift(x, smin=-5, smax=5):
    shift = np.around(random.uniform(a=smin, b=smax))
    return x + shift

def DA_Scale(x, smin=0.8, smax=1.2):
    scale = random.uniform(a=smin, b=smax)
    return np.round(x*scale)

def DA_TimeWarp(X, sigma=0.2):
    length = X.shape[0]
    channel = X.shape[1]
    knot = 4
    
    from scipy.interpolate import CubicSpline      # for warping

    xx = (np.ones((channel,1))*(np.arange(0, length, (length-1)/(knot+1)))).transpose()
    yy = np.random.normal(loc=1.0, scale=sigma, size=(knot+2, channel))
    x_range = np.arange(length)
    cs_x = CubicSpline(xx[:,0], yy[:,0])
    tt = np.array([cs_x(x_range)]).transpose()

    tt_new = np.cumsum(tt, axis=0)        # Add intervals to make a cumulative graph
    # Make the last value to have X.shape[0]
    t_scale = [(length-1)/tt_new[-1,0]]
    tt_new[:,0] = tt_new[:,0]*t_scale[0]

    # tt_new = DistortTimesteps(X, sigma)
    X_new = np.zeros(X.shape)
    x_range = np.arange(length)
    
    X_new[:,0] = np.interp(x_range, tt_new[:,0], X[:,0])
    X_new[:,1] = np.interp(x_range, tt_new[:,0], X[:,1])
    return X_new

def data_preprocess(x,aug_func=[], prob=0.5, normalized=True):
    do_or_not = Probability(prob)
    
    length = x.shape[0]
    # get x and then remove zeros (no info)
    x = x[(x[:,0] > 0.0) * (x[:,1] > 0.0)]
    
    # add random_noise
    if aug_func:
        for func in aug_func:
            if do_or_not():
                x = func(x)
    
    if normalized:
        x[:,0] = (x[:,0] - MEAN_B)/STD_B
        x[:,1] = (x[:,1] - MEAN_M)/STD_M

        
        # x1, x2 = np.mean(x, axis=0)
#         noise = np.array([[random.gauss(mu=0, sigma=0.01), 
#                            random.gauss(mu=0, sigma=0.01)] for _ in range(x.shape[0])], dtype=np.float32)
#         x = x + noise

    # transpose to (n_channel, arbitrary length), then padd to (n_channel, length)
    x = pad_sequences(np.transpose(x), padding='post', value=0.0, maxlen=length, dtype=np.float)

    # transpose back to original shape and store
    return np.transpose(x)


def k_slice_X(Xvalid, Yvalid, k_slice=5, length=300, class_weight = {}):
    """
    # moving across a sequence, we slice out "k_slice" segments with a constant interval
    # in order to increase validation data
    # ex:  |------------------|
    # 1    |------|
    # 2       |------|
    # 3          |------|
    # 4             |------|
    # 5                |------|
    """
    if not class_weight:
        class_weight = dict()
        for i in range(Yvalid.shape[1]):
            class_weight[i] = 1

    intvl = (Xvalid.shape[1] - length)//k_slice

    Xtest = np.empty((Xvalid.shape[0]*k_slice, length, Xvalid.shape[2]))
    Ytest = np.empty((Yvalid.shape[0]*k_slice, Yvalid.shape[1]))
    Wtest = np.empty((Yvalid.shape[0]*k_slice,))

    for k in range(k_slice):
        st = k * Xvalid.shape[0]
        for i in range(Xvalid.shape[0]):
            # print(st+i)
            Xtest[st+i,:,:] = data_preprocess(Xvalid[i,k*intvl:(k*intvl+length),:])
            Ytest[st+i,:] = Yvalid[i,:]
            Wtest[st+i] = class_weight[np.argmax(Yvalid[i,:])]
    return Xtest, Ytest, Wtest

def get_n_zeros(d):
    n_zeros = list()
    for i in range(d.shape[0]):
        n_zeros.append(sum(d[i,:] ==0))
    return np.array(n_zeros)

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

def plot_keras_csv_logger(csv_logger, save_dir='', accuracy=False):
    loss = pd.read_table(csv_logger.filename, delimiter=',')
    print('min val_loss {0} at epoch {1}'.format(min(loss.val_loss), np.argmin(loss.val_loss)))
    plt.plot(loss.epoch, loss.loss, label='loss')
    plt.plot(loss.epoch, loss.val_loss, label='val_loss')
    plt.legend()
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.savefig(os.path.join(save_dir, 'loss.png'))
    plt.close()

    if accuracy:
        print('max val_accu {0} at epoch {1}'.format(max(loss.val_acc), np.argmax(loss.val_acc)))
        plt.plot(loss.epoch, loss.acc, label='accu')
        plt.plot(loss.epoch, loss.val_acc, label='val_accu')
        plt.legend()
        plt.xlabel('epoch')
        plt.ylabel('accuracy')
        plt.savefig(os.path.join(save_dir, 'accu.png'))
        plt.close()

/usr/local/lib/python3.5/dist-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


### Data preparation

In [6]:
data_dir = "/data/put_data/cmchang/gynecology/data/"
d = pd.read_csv(os.path.join(data_dir, 'data_merged.csv'))

# ['variability', 'deceleration', 'management', 'UA']

FLAG_target = 'variability'
n_classes = len(set(d[FLAG_target]))

In [7]:
acceptable_zeros_threshold = 90
d = d[get_n_zeros(np.array(d[[k for k in d.columns if 'b-' in k]], dtype=np.float)) <= acceptable_zeros_threshold]

In [8]:
for k in d.columns:
    if 'b-' in k or 'm-' in k:
        print(k, end='\r')
        d.loc[d[k]==0, k] = np.nan

In [11]:
from sklearn.model_selection import train_test_split
train_id, valid_id = train_test_split(list(set(d.ID)), test_size=0.3, random_state=13)

train_d, valid_d = d[[k in set(train_id) for k in d.ID]], d[[k in set(valid_id) for k in d.ID]]

In [12]:
train_db = np.array(train_d[[k for k in train_d.columns if 'b-' in k]].interpolate(limit_direction='both', axis=1), dtype=np.float)
train_dm = np.array(train_d[[k for k in train_d.columns if 'm-' in k]].interpolate(limit_direction='both', axis=1), dtype=np.float)

valid_db = np.array(valid_d[[k for k in valid_d.columns if 'b-' in k]].interpolate(limit_direction='both', axis=1), dtype=np.float)
valid_dm = np.array(valid_d[[k for k in valid_d.columns if 'm-' in k]].interpolate(limit_direction='both', axis=1), dtype=np.float)

In [13]:
Xtrain = np.stack([train_db, train_dm], axis=2)
Xvalid = np.stack([valid_db, valid_dm], axis=2)

In [14]:
Ytrain = keras.utils.to_categorical(np.array(train_d[FLAG_target]), num_classes=n_classes)
Yvalid = keras.utils.to_categorical(np.array(valid_d[FLAG_target]), num_classes=n_classes)

In [ ]:
# n_rep = 5
# aug_Xtrain = np.empty((Xtrain.shape[0] * n_rep, Xtrain.shape[1], Xtrain.shape[2]), dtype=np.float32)
# for i in range(Xtrain.shape[0]):
#     for j in range(n_rep):
#         aug_Xtrain[i*n_rep+j, :, : ] = DA_TimeWarp(Xtrain[i,:,:], sigma=0.2)
        
# aug_Ytrain = np.repeat(Ytrain, repeats=n_rep, axis=0)

# aug_Xtrain = np.concatenate([Xtrain, aug_Xtrain])
# aug_Ytrain = np.concatenate([Ytrain, aug_Ytrain])

In [15]:
# Convolution
kernel_size = 3
filters = 64
strides = 1
layers = 10
activation='relu'
kernel_initializer = 'RandomNormal'

# input, output
FLAG_batch_size = 32
FLAG_length = 300
FLAG_n_channel = 2

# Training
lr_rate = 1e-5

In [16]:
from sklearn.utils.class_weight import compute_class_weight

y_integers = np.argmax(Ytrain, axis=1)
d_class_weight = compute_class_weight('balanced', np.unique(y_integers), y_integers)
class_weight = dict(enumerate(d_class_weight))
print(class_weight)

{0: 0.5694444444444444, 1: 4.1}


In [ ]:
# class_weight = {0: 1.0, 1: #4.0}

In [ ]:
# from collections import Counter
# class_weight = dict()
# for k, v in dict(Counter(train_d[FLAG_target])).items():
#     class_weight[k] = min(train_d.shape[0]//v, 25)
# print(class_weight)

In [17]:
# moving across a sequence, we slice out "k_slice" segments with a constant interval
# in order to increase validation data
"""
# ex:  |------------------|
# 1    |------|
# 2       |------|
# 3          |------|
# 4             |------|
# 5                |------|
"""


k_slice = 5
intvl = (Xvalid.shape[1] - FLAG_length)//5

Xtest = np.empty((Xvalid.shape[0]*k_slice, FLAG_length, Xvalid.shape[2]))
Ytest = np.empty((Yvalid.shape[0]*k_slice, Yvalid.shape[1]))
Wtest = np.empty((Yvalid.shape[0]*k_slice,))

for k in range(k_slice):
    st = k * Xvalid.shape[0]
    for i in range(Xvalid.shape[0]):
        # print(st+i)
        Xtest[st+i,:,:] = data_preprocess(Xvalid[i,k*intvl:(k*intvl+FLAG_length),:])
        Ytest[st+i,:] = Yvalid[i,:]
        Wtest[st+i] = class_weight[np.argmax(Yvalid[i,:])]
    
print('train: {0}, valid: {1}'.format(Xtrain.shape, Xtest.shape))

train: (123, 600, 2), valid: (270, 300, 2)


In [18]:
print("===== create directory =====")
model_id = FLAG.target + "_" + datetime.now().strftime("%y%m%d%H%M%S")
model_save = os.path.join(FLAG.model_save, FLAG.target, model_id)
summary_save = os.path.join(FLAG.model_save, FLAG.target, 'summary_'+FLAG.target+'.csv')

if not os.path.exists(model_save):
    os.makedirs(model_save)
    print(model_save)


FLAG_model_save = os.path.join('/data/put_data/cmchang/gynecology/model/', FLAG_target+'_test')
if not os.path.exists(FLAG_model_save):
    os.mkdir(FLAG_model_save)
    print('directory {0} is created.'.format(FLAG_model_save))
else:
    print('directory {0} already exists.'.format(FLAG_model_save))

directory /data/put_data/cmchang/gynecology/model/variability_test already exists.


In [ ]:
def my_generator(Xtrain, Ytrain, length=300, n_channel=2, n_classes=2, aug_func=[], prob=0.5, batch_size=16):
    n_sample = Xtrain.shape[0]
    n_length = Xtrain.shape[1]
    ind = list(range(n_sample))
    x = np.empty((batch_size, length, n_channel), dtype=np.float)
    y = np.empty((batch_size, n_classes), dtype=int)

    while True:
        np.random.shuffle(ind)
        for i in range(n_sample//batch_size):
            st = random.choice(np.arange(0, Xtrain.shape[1] - length))
            i_batch = ind[i*batch_size:(i+1)*batch_size]
            for j, k in enumerate(i_batch):
                x[j,:] = data_preprocess(Xtrain[k,st:(st+length),:], aug_func=aug_func, prob=prob)
                y[j,:] = Ytrain[k,:]
            yield x, y

###  Model

In [ ]:
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.optimizers import Adam, SGD, Adamax

In [ ]:
model = build_model(length=FLAG_length, n_channel=FLAG_n_channel, n_classes=n_classes, filters=filters, kernel_size=kernel_size, layers=layers,
                   activation=activation, kernel_initializer=kernel_initializer, l_2=0.0)

In [ ]:
model.summary()

In [ ]:
adam = Adamax(lr=lr_rate, beta_1=0.5, beta_2=0.999, epsilon=1e-08, decay = 0.0)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

#### Declare DataGenerator for Keras

In [ ]:
csv_logger = keras.callbacks.CSVLogger(os.path.join(FLAG_model_save, 'training.log'))
checkpoint = keras.callbacks.ModelCheckpoint(os.path.join(FLAG_model_save, 'model.h5'), 
                                             monitor='val_loss', 
                                             verbose=1, 
                                             save_best_only=True,
                                             save_weights_only=False,
                                             mode='min',
                                             period=1)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor = 0.5, patience = 5, min_lr = 0, cooldown = 5, verbose = True)

In [ ]:
earlystop = EarlyStopping(monitor='val_loss', patience=30, min_delta=1e-4)

In [ ]:
# Train model on dataset
model.fit_generator(generator=my_generator(Xtrain, Ytrain, 
                                           #aug_func=[DA_Shift, DA_Scale],
                                           length=FLAG_length, 
                                           n_channel=FLAG_n_channel, 
                                           n_classes=n_classes,
                                           #prob=0.25
                                          ),
                    class_weight=class_weight,
                    validation_data=(Xtest, Ytest, Wtest),
                    steps_per_epoch=50, 
                    epochs=150,
                    callbacks=[csv_logger,
                               reduce_lr, 
                               checkpoint,
                              earlystop])

In [ ]:
loss = pd.read_table(csv_logger.filename, delimiter=',')

plt.plot(loss.epoch, loss.loss, label='loss')
plt.plot(loss.epoch, loss.val_loss, label='val_loss')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss (cross entropy)')
# plt.xlim([1,50])
# plt.ylim([0,2])
plt.savefig(os.path.join(FLAG_model_save, 'loss.png'))

In [ ]:
loss = pd.read_table(csv_logger.filename, delimiter=',')

plt.plot(loss.epoch, loss.acc, label='accu')
plt.plot(loss.epoch, loss.val_acc, label='val_accu')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.savefig(os.path.join(FLAG_model_save, 'accu.png'))

In [ ]:
from keras.models import load_model
trained_model = load_model(os.path.join(FLAG_model_save,'model.h5'))

In [ ]:
Pred = trained_model.predict(Xtest)

In [ ]:
ypred_aug = np.argmax(Pred,axis=1)
ytest_aug = np.argmax(Ytest,axis=1)

In [ ]:
ypred = (np.mean(ypred_aug.reshape(5,-1), axis=0) > 0.5) + 0 # voting
# ypred = np.argmax(np.mean(ypred_aug.reshape(5, 84, 2), axis=0), axis=1)
ytest = np.argmax(Yvalid, axis=1)

In [ ]:
from collections import Counter
Counter(np.mean(ypred_aug.reshape(5,-1), axis=0))

In [ ]:
Counter(np.mean(ypred_aug.reshape(5,-1), axis=0)[ypred != ytest])

#### save wrong prediction plots

In [ ]:
if not os.path.exists(os.path.join(FLAG_model_save, 'wrong/')):
    os.mkdir(os.path.join(FLAG_model_save, 'wrong/'))
    print('{0} created'.format(os.path.join(FLAG_model_save, 'wrong/')))

In [ ]:
mom  = valid_dm[ypred != ytest]
baby = valid_db[ypred != ytest]

for i in range(mom.shape[0]):
    key = np.array(valid_d.key)[(ypred != ytest)][i]
    fig = plt.figure(figsize=(6,6))
    fig.subplots_adjust(hspace=0.4, wspace=0.4)
    
    plt.subplot(2, 1, 1)
    plt.plot(mom[i,:], '.-', color='red')
    plt.ylabel('Mom')
    plt.title('pred={0}, true={1}'.format( ypred[(ypred != ytest)][i], ytest[(ypred != ytest)][i]))

    plt.subplot(2, 1, 2)
    plt.plot(baby[i,:], '.-')
    plt.xlabel('time (s)')
    plt.ylabel('Baby')
    plt.title('key={0}'.format(key))

    plt.savefig(os.path.join(FLAG_model_save, 'wrong/'+key+'.png'))
    plt.show()
    plt.close()

In [ ]:
mom  = valid_dm[(ypred + ytest) == 2]
baby = valid_db[(ypred + ytest) == 2]

for i in range(mom.shape[0]):
    fig = plt.figure()
    fig.subplots_adjust(hspace=0.4, wspace=0.4)
    
    plt.subplot(2, 1, 1)
    plt.plot(mom[i,:], '.-', color='red')
    plt.ylabel('Mom')
    plt.title('n_zeros = {0}'.format(get_n_zeros(mom[i:(i+1),:])[0]))

    plt.subplot(2, 1, 2)
    plt.plot(baby[i,:], '.-')
    plt.xlabel('time (s)')
    plt.ylabel('Baby')
    plt.title('n_zeros = {0}'.format(get_n_zeros(baby[i:(i+1),:])[0]))

    plt.show()
    plt.close()
    # plt.savefig(os.path.join(data_dir, '../eda/'+key[i]+'.png'))type=np.float)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
print(classification_report(y_pred=ypred, y_true=ytest))

In [ ]:
print(classification_report(y_pred=ypred_aug, y_true=ytest_aug))

In [ ]:
cfm = confusion_matrix(y_pred=ypred, y_true=ytest)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cfm, classes=np.arange(n_classes),
                      title='Confusion matrix, without normalization')
plt.savefig(os.path.join(FLAG_model_save, 'voting_confusion_matrix.png'))
plt.close()

In [ ]:
cfm = confusion_matrix(y_pred=ypred_aug, y_true=ytest_aug)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cfm, classes=np.arange(n_classes),
                      title='Confusion matrix, without normalization')
plt.savefig(os.path.join(FLAG_model_save, 'segment_confusion_matrix.png'))
plt.close()

#### t-SNE

In [ ]:
feature_name = ''
for l in trained_model.layers:
    if 'global_average_pooling' in l.name:
        feature_name = l.name
if not feature_name:
    print('global_average_pooling layer not found')

In [ ]:
features = trained_model.get_layer(feature_name)

In [ ]:
extractor = Model(inputs=trained_model.input, outputs=features.output)

In [ ]:
valid_feature_list = list()
valid_feature_list.append(extractor.predict(x=Xtest))

In [ ]:
Fvalid = np.array(valid_feature_list[0])

In [ ]:
from sklearn.manifold import TSNE

F_embedded = TSNE(n_components=2, perplexity=100).fit_transform(Fvalid)

In [ ]:
labels = np.argmax(Ytest,axis=1)
color = ['#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00']

In [ ]:
# F_embedded
for l in (set(labels)):
    plt.scatter(F_embedded[list(np.where(labels==l)[0]),0], F_embedded[list(np.where(labels==l)[0]),1],
                color=color[l], alpha=0.8)
plt.xlabel('tSNE-0')
plt.ylabel('tSNE-1')
legend=plt.legend(title='Altitude', fontsize=14)
